In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import json
import spacy

In [3]:
nlp = spacy.load("en_core_web_lg")

In [4]:
annotation_array = []

read_annotation = open("./annotated data/NLP Project.json")
for line in read_annotation:
    data = json.loads(line)
    annotation_array.append(data)

In [5]:
len(annotation_array)

264

In [6]:
annotation_array[2]["content"]

'Bake for 5 minutes until set but not browned.'

In [7]:
for annotated_text in annotation_array[2]["annotation"]:
    print(annotated_text["points"][0]["start"])
    print(annotated_text["points"][0]["end"])
    print(annotated_text["points"][0]["text"])
    print(annotated_text["label"][0])

37
43
browned
U-Action
0
3
Bake
U-Action


<IPython.core.display.Javascript object>

In [8]:
preprocessed_df = pd.DataFrame({"sentence#": [], "word": [], "pos": [], "tag": []})

<IPython.core.display.Javascript object>

In [9]:
for i, annotation in enumerate(annotation_array):
    doc = nlp(annotation["content"])

    for token in doc:

        found = False

        # some content does not have annotations
        if annotation["annotation"] is not None:
            for annotated_text in annotation["annotation"]:

                # matches with text and the start and end character offset then add the data to df
                if (
                    (token.text == annotated_text["points"][0]["text"])
                    and (token.idx == annotated_text["points"][0]["start"])
                    and (
                        (token.idx + token.__len__() - 1)
                        == annotated_text["points"][0]["end"]
                    )
                ):

                    # add the token with the tag
                    preprocessed_df = preprocessed_df.append(
                        pd.DataFrame(
                            {
                                "sentence#": [i],
                                "word": [token.text],
                                "pos": [token.tag_],
                                "tag": [annotated_text["label"][0]],
                            }
                        )
                    )

                    # set found to true
                    found = True

                    # break since we found the token
                    break

        if not found:
            # add the O tag
            preprocessed_df = preprocessed_df.append(
                pd.DataFrame(
                    {
                        "sentence#": [i],
                        "word": [token.text],
                        "pos": [token.pos_],
                        "tag": ["O"],
                    }
                )
            )

<IPython.core.display.Javascript object>

In [10]:
preprocessed_df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

In [11]:
preprocessed_df.head()

sentence#     word   pos        tag
0        0.0  Preheat    VB   U-Action
1        0.0     oven    NN  U-Utensil
2        0.0       to   ADP          O
3        0.0      425   NUM          O
4        0.0  degrees  NOUN          O

<IPython.core.display.Javascript object>

In [12]:
preprocessed_df["pos"].unique()

array(['VB', 'NN', 'ADP', 'NUM', 'NOUN', 'DET', 'CCONJ', 'PUNCT', 'PART',
       'VBN', 'ADJ', 'NNS', 'ADV', 'PROPN', 'VERB', 'SYM', 'AUX', 'JJ',
       'VBP', 'SCONJ', 'PRON', 'NNP', 'VBG', 'VBD', 'RB', 'RP', 'VBZ',
       'INTJ'], dtype=object)

<IPython.core.display.Javascript object>

In [13]:
preprocessed_df.to_csv("./tagged_data.csv")

<IPython.core.display.Javascript object>